In [30]:
%load_ext autoreload
%autoreload 2

import os
import oddsapi
import requests
import joblib
import pandas as pd
import numpy as np
from common.utils.odds_utils import get_tournament_winner_odds, get_group_stage_odds, get_group_stage_OU
from common.utils.holdet_utils import get_holdet_data, get_play_off_opps, get_rw
from common.utils.simulation_utils import WorldCupTeam, WorldCupSim
from common.utils.group_stage import get_group_stage, playoffs
from common.utils.dict_utils import eng2dan
from scipy.optimize import fsolve
import warnings
warnings.filterwarnings('ignore')
ROUND = 1
API_KEY = os.getenv("API_KEY")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# load data
df_holdet = get_holdet_data()
df_winner = get_tournament_winner_odds(API_KEY)
df_group_stage = get_group_stage_odds(API_KEY)
df_get_group_stage_OU = get_group_stage_OU(API_KEY)
df_groups = pd.read_csv("common/ressources/df_groups.csv")[["Team","group"]]
df_group_stage = df_group_stage[['home_team','away_team','prob1','probx', 'prob2']].merge(df_groups.rename(columns =  {"Team":"home_team"}), on = "home_team")

In [32]:
_df_group_stage = get_group_stage(df_get_group_stage_OU.copy(), df_group_stage.copy(), ROUND, eng2dan)

In [33]:
df_rw = get_rw(df_holdet)
df_rw.rename(columns = {"Player_match": "name1"}, inplace = True)
print(df_holdet.shape)
df_holdet = df_holdet.merge(df_rw, on = "name1", how ="left")
print(df_holdet.shape)
df_holdet = df_holdet[['name1', 'name2', 'country', 'price', 'position', 'Player', 'Team', 'Opp', 'Pos', 'MIN','G', 'A', 'SOG', 'Y','R']]
print(df_holdet.shape)

(835, 10)
(835, 39)
(835, 15)


In [34]:
for round in [str(_) for _ in range(ROUND,4)]:
    df_temp1 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["home_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"home_team":"country"}), on = "country")
    df_temp1.rename(columns = {"prob1":f"prob_winning_{round}","probx":f"prob_draw_{round}","prob2":f"prob_lose_{round}","xGH":f"E_goals_{round}","xGA":f"E_goals_against_{round}"}, inplace = True)

    df_temp2 = df_holdet[["name1","name2","country"]].merge(_df_group_stage.loc[_df_group_stage["round"]==round,["away_team","prob1","probx","prob2","xGH","xGA"]].rename(columns = {"away_team":"country"}), on = "country")
    df_temp2.rename(columns = {"prob1":f"prob_lose_{round}","probx":f"prob_draw_{round}","prob2":f"prob_winning_{round}","xGA":f"E_goals_{round}","xGH":f"E_goals_against_{round}"}, inplace = True)

    df_temp_temp = pd.concat([df_temp1,df_temp2],axis=0)
    df_holdet = df_holdet.merge(df_temp_temp, on = ["name1","name2","country"])

In [38]:
# df_winner  = playoffs(df_winner.copy(),_df_group_stage.copy(), eng2dan)
# df_holdet = df_holdet.merge(df_winner[['country','R16_prob', 'QF_prob', 'SF_prob', 'Final_prob', 'W_prob', \
#                                        'prob_to_win_R16', 'prob_to_win_QF', 'prob_to_win_SF','prob_to_win_Final', \
#                                        'xG_R16', 'xG_QF', 'xG_SF', 'xG_Final','xGa_R16', 'xGa_QF', 'xGa_SF', 'xGa_Final']], on = "country")

df_holdet["selected"] = ~df_holdet['G'].isna()

In [55]:
for country in df_holdet["country"].unique():
    xG_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float).sum()
    xA_sum = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float).sum()
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'G'].astype(float)/xG_sum
    df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A_share'] = df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),'A'].astype(float)/xA_sum

    df_holdet.loc[(df_holdet["country"]==country) & (~df_holdet["selected"]),"Exptectred_points"] = -5000
    for player in df_holdet.loc[(df_holdet["country"]==country) & (df_holdet["selected"]),"name1"]:
        points = 0
        goal_share = df_holdet.loc[df_holdet["name1"]==player,"G_share"].values[0]
        assist_share = df_holdet.loc[df_holdet["name1"]==player,"A_share"].values[0]
        yellow_prob = df_holdet.loc[df_holdet["name1"]==player,'Y'].astype(float).values[0]
        red_prob = df_holdet.loc[df_holdet["name1"]==player,'R'].astype(float).values[0]
        # group games
        loop = 4
        for i in range(ROUND,loop):
            points += 7000**(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_winning_{i}'].values[0]*25000**(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_draw_{i}'].values[0]*5000**(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'prob_lose_{i}'].values[0]*-15000**(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*10000**(1/(i-ROUND+1))
            points += df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0]*-10000**(1/(i-ROUND+1))
            points += yellow_prob * -20000**(1/(i-ROUND+1))
            points += yellow_prob*yellow_prob*-20000**(1/(i-ROUND+1))
            points += red_prob * -50000**(1/(i-ROUND+1))
            if i == loop:
                for idx, game in enumerate(["R16","QF","SF","Final"]):
                    prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                    factor = 1/(2*(loop+idx))
                    points += 7000**factor
                    points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*-10000**factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0]*25000**factor
                    points += prob*(1-df_holdet.loc[df_holdet["name1"]==player,f'prob_to_win_{game}'].values[0])*-15000**factor
                    points += prob*yellow_prob * -20000**factor
                    points += prob*yellow_prob*yellow_prob*-20000**factor
                    points += prob*df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0]*10000**factor
                    points += prob*red_prob * -50000**factor
                    
            if df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Mål":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*75000**(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xGa_{game}'].values[0])*75000**factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Forsvar":
                points += np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'E_goals_against_{i}'].values[0])*50000**(1/(i-ROUND+1))
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*175000**(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000**(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*50000**factor
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*175000**factor
                        points += prob*assist_share**np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000**factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Midtbane":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*150000**(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000**(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*150000**factor
                        points += prob*assist_share**np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000**factor
                        
            elif df_holdet.loc[df_holdet["name1"]==player,"position"].values[0]=="Angreb":
                points += goal_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*125000**(1/(i-ROUND+1))
                points += assist_share*df_holdet.loc[df_holdet["name1"]==player,f'E_goals_{i}'].values[0]*60000**(1/(i-ROUND+1))
                if i == loop:
                    for idx, game in enumerate(["R16","QF","SF","Final"]):
                        prob = df_holdet.loc[df_holdet["name1"]==player,f'{game}_prob'].values[0]
                        factor = 1/(2*(loop+idx))
                        points += prob*goal_share*np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*125000**factor
                        points += prob*assist_share**np.exp(-df_holdet.loc[df_holdet["name1"]==player,f'xG_{game}'].values[0])*60000**factor
            if i == ROUND:
                df_holdet.loc[df_holdet["name1"]==player,"E_points_round1"] = points
        df_holdet.loc[df_holdet["name1"]==player,"E_points_total"] = points

In [61]:
df_holdet[df_holdet["name1"]=="Seyed Hossein Hosseini"]

,name1,name2,country,price,position,Player,Team,Opp,Pos,MIN,...,xGa_R16,xGa_QF,xGa_SF,xGa_Final,selected,G_share,A_share,Exptectred_points,E_points_round1,E_points_total
767,Seyed Hossein Hosseini,S. Hosseini,Iran,2500000.0,Mål,Seyed Hossein Hosseini,IRN,ENG,G,90,...,0.794852,0.060832,0.156147,0.156147,True,0.0,0.0,NaN,824845.210625,827004.522337


In [62]:
df_holdet[["name1","country","E_points_round1","position"]].sort_values("E_points_round1", ascending =False).iloc[0:40]

,name1,country,E_points_round1,position
767,Seyed Hossein Hosseini,Iran,824845.210625,Mål
526,Mohammed Al-Owais,Saudi-Arabien,805339.718853,Mål
798,Esteban Alvarado,Costa Rica,737519.928809,Mål
716,Aymen Dahmen,Tunesien,675405.029260,Mål
343,Vanja Milinkovic-Savic,Serbien,674784.883032,Mål
735,Mathew Ryan,Australien,629214.818046,Mål
643,Dayne St. Clair,Canada,628743.285208,Mål
484,Daniel Schmidt,Japan,627500.590541,Mål
261,Edouard Mendy,Senegal,580887.502555,Mål
550,Yassine 'Bono' Bounou,Marokko,527270.756021,Mål
